In [1]:
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from models import GCN
import importlib
import utils
importlib.reload(utils)
from utils import *

# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)


In [2]:
# get paths to available grahs metadata
N_sample = 200000
adj_mat, features_mat, train_labels_mat, test_labels_mat, val_labels_mat, train_mask_mat, test_mask_mat, val_mask_mat = load_data("txt_graph2216_21012020", N_sample)

In [3]:
print(adj_mat.shape)
print(features_mat.shape)
print(train_labels_mat.shape)
print(test_labels_mat.shape)
print(val_labels_mat.shape)
print(train_mask_mat.shape)
print(test_mask_mat.shape)
print(val_mask_mat.shape)

print("number of masked elements")
print("train: " + str(len(np.where(train_mask_mat[:,1]==1)[0])))
print("test: " + str(len(np.where(test_mask_mat[:,1]==1)[0])))
print("val: " + str(len(np.where(val_mask_mat[:,1]==1)[0])))

print("\n positions of ones in masked labels")
print(np.where(train_labels_mat[np.array(train_mask_mat[:,1],dtype=bool)]==1)[1])
print(np.where(test_labels_mat[np.array(test_mask_mat[:,1],dtype=bool)]==1)[1])
print(np.where(val_labels_mat[np.array(val_mask_mat[:,1],dtype=bool)]==1)[1])

print("\n ones outside the mask")
print(np.where(train_labels_mat[np.logical_not(np.array(train_mask_mat[:,1],dtype=bool))]==1))
print(np.where(test_labels_mat[np.logical_not(np.array(test_mask_mat[:,1],dtype=bool))]==1))
print(np.where(val_labels_mat[np.logical_not(np.array(val_mask_mat[:,1],dtype=bool))]==1))

#sum_global_nodes = np.sum(adj_mat.toarray()[:,np.array(train_mask_mat[:,1],dtype=bool)], axis = 1) + np.sum(adj_mat.toarray()[:,np.array(test_mask_mat[:,1],dtype=bool)], axis = 1) + np.sum(adj_mat.toarray()[:,np.array(val_mask_mat[:,1],dtype=bool)], axis = 1)
#print(adj_mat.shape[1] - sum(sum_global_nodes))

(465542, 465542)
(465542, 100)
(465542, 5)
(465542, 5)
(465542, 5)
(465542, 5)
(465542, 5)
(465542, 5)
number of masked elements
train: 779
test: 668
val: 778

 positions of ones in masked labels
[2 1 0 4 0 4 4 1 3 0 3 3 0 4 2 0 0 1 4 1 1 3 0 2 0 1 1 0 4 0 4 3 3 2 2 0 2
 2 4 4 1 3 2 2 0 4 4 4 3 1 2 0 3 1 2 0 1 3 4 0 3 3 3 4 0 1 4 1 3 4 3 1 1 1
 3 0 0 4 3 2 1 0 0 0 2 0 4 3 0 0 4 4 3 2 3 3 1 4 3 4 1 0 3 2 4 3 4 4 0 1 4
 3 1 0 1 3 3 4 4 3 3 3 4 3 2 3 1 2 3 1 3 0 2 0 3 2 4 2 0 2 0 4 2 0 2 1 0 2
 0 0 3 0 1 2 3 3 3 1 4 3 2 4 3 3 0 4 3 1 3 1 3 0 4 2 4 2 0 3 2 3 2 2 2 0 2
 2 4 0 2 3 0 2 0 0 0 2 0 0 2 2 2 3 2 0 3 1 4 3 0 0 4 1 2 2 4 0 2 3 0 4 2 3
 0 0 2 1 0 0 3 3 3 1 2 4 0 3 0 3 3 1 4 3 2 4 2 3 4 0 3 4 0 3 3 4 0 0 0 3 3
 0 2 1 4 4 3 0 0 4 4 1 1 4 3 0 3 4 3 2 3 2 1 4 3 4 0 3 0 4 4 0 4 1 2 2 3 3
 3 0 2 4 3 3 2 3 4 2 1 0 2 4 4 1 4 3 4 3 1 3 3 1 3 2 0 4 1 0 2 3 0 4 4 4 1
 3 3 4 1 0 4 2 3 3 0 0 1 2 3 1 0 4 0 2 3 1 1 0 2 3 3 1 0 4 3 3 2 4 0 0 1 1
 0 0 0 2 0 0 3 3 4 3 1 1 0 4 2 1 0 0 3 1 4 1 3 0 3 1 2

In [4]:
#delete all flags before declaration new one
del_all_flags(tf.flags.FLAGS)

# Settings
flags = tf.app.flags
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('mode', '', 'kernel') # for line by line mode
tf.app.flags.DEFINE_string('port', '', 'kernel') # for line by line mode
tf.app.flags.DEFINE_string('f', '', 'kernel') # for jupyter notebook

flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.') #0.01
flags.DEFINE_integer('epochs', 2000, 'Number of epochs to train.') #200
flags.DEFINE_integer('hidden1', 64, 'Number of units in hidden layer 1.') #16
flags.DEFINE_float('dropout', 0.8, 'Dropout rate (1 - keep probability).') #0.5
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 2000, 'Tolerance for early stopping (# of epochs).') #10

# Some preprocessing
features = preprocess_features(features_mat)
support = [preprocess_adj(adj_mat)]
num_supports = 1
model_func = GCN 

/home/robertb/python/GCN_text_classif/gcn/utils.py:57: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


In [5]:
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, train_labels_mat.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
# Initialize session
sess = tf.Session()
# Init variables
sess.run(tf.global_variables_initializer())

In [ ]:
# Train model
cost_val = []

for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, train_labels_mat, train_mask_mat[:,1], placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, val_mask_mat, val_mask_mat[:,1], placeholders, sess, model)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

Epoch: 0001 train_loss= 30.33150 train_acc= 0.22721 val_loss= 39.41757 val_acc= 0.72237 time= 10.71388
Epoch: 0002 train_loss= 16.81936 train_acc= 0.20924 val_loss= 37.68326 val_acc= 0.70823 time= 10.01661
Epoch: 0003 train_loss= 26.68737 train_acc= 0.22593 val_loss= 36.66272 val_acc= 0.69280 time= 10.01657
Epoch: 0004 train_loss= 28.54598 train_acc= 0.21823 val_loss= 35.95023 val_acc= 0.66838 time= 10.07508
Epoch: 0005 train_loss= 20.57083 train_acc= 0.22721 val_loss= 35.04095 val_acc= 0.64910 time= 10.03505
Epoch: 0006 train_loss= 23.62959 train_acc= 0.21566 val_loss= 34.01050 val_acc= 0.62982 time= 10.04319
Epoch: 0007 train_loss= 25.53231 train_acc= 0.19384 val_loss= 32.79931 val_acc= 0.62725 time= 10.14809
Epoch: 0008 train_loss= 23.18822 train_acc= 0.23363 val_loss= 31.68569 val_acc= 0.61954 time= 9.98431
Epoch: 0009 train_loss= 21.49719 train_acc= 0.21053 val_loss= 30.61210 val_acc= 0.61440 time= 10.07334
Epoch: 0010 train_loss= 33.64248 train_acc= 0.19384 val_loss= 29.52277 val

Epoch: 0081 train_loss= 19.02888 train_acc= 0.22080 val_loss= 30.96159 val_acc= 0.34190 time= 10.09306
Epoch: 0082 train_loss= 20.47910 train_acc= 0.22721 val_loss= 30.85707 val_acc= 0.34319 time= 10.16634
Epoch: 0083 train_loss= 12.13223 train_acc= 0.21823 val_loss= 30.75383 val_acc= 0.34190 time= 10.56029
Epoch: 0084 train_loss= 19.33172 train_acc= 0.20924 val_loss= 30.67305 val_acc= 0.34319 time= 10.25206
Epoch: 0085 train_loss= 13.63630 train_acc= 0.19255 val_loss= 30.61614 val_acc= 0.34319 time= 10.09942
Epoch: 0086 train_loss= 18.40340 train_acc= 0.21053 val_loss= 30.58290 val_acc= 0.34319 time= 10.02798
Epoch: 0087 train_loss= 16.76731 train_acc= 0.18870 val_loss= 30.56355 val_acc= 0.34190 time= 10.00977
Epoch: 0088 train_loss= 15.22830 train_acc= 0.22465 val_loss= 30.48252 val_acc= 0.33933 time= 10.10460
Epoch: 0089 train_loss= 21.75092 train_acc= 0.21053 val_loss= 30.35401 val_acc= 0.33548 time= 10.02415
Epoch: 0090 train_loss= 16.74750 train_acc= 0.21823 val_loss= 30.27246 va

Epoch: 0161 train_loss= 10.67792 train_acc= 0.24647 val_loss= 21.90744 val_acc= 0.33162 time= 10.01205
Epoch: 0162 train_loss= 15.27615 train_acc= 0.24005 val_loss= 21.80357 val_acc= 0.33548 time= 10.04854
Epoch: 0163 train_loss= 11.57002 train_acc= 0.20924 val_loss= 21.73407 val_acc= 0.33290 time= 10.13722
Epoch: 0164 train_loss= 9.30929 train_acc= 0.24647 val_loss= 21.67608 val_acc= 0.33033 time= 10.09720
Epoch: 0165 train_loss= 15.22788 train_acc= 0.22721 val_loss= 21.63949 val_acc= 0.32391 time= 10.04852
Epoch: 0166 train_loss= 13.88878 train_acc= 0.21053 val_loss= 21.59272 val_acc= 0.32134 time= 10.00591
Epoch: 0167 train_loss= 8.65047 train_acc= 0.22080 val_loss= 21.55916 val_acc= 0.32262 time= 10.02684
Epoch: 0168 train_loss= 17.35087 train_acc= 0.21694 val_loss= 21.51568 val_acc= 0.32391 time= 9.99444
Epoch: 0169 train_loss= 11.91391 train_acc= 0.21181 val_loss= 21.51903 val_acc= 0.32519 time= 10.26238
Epoch: 0170 train_loss= 10.18093 train_acc= 0.24775 val_loss= 21.51010 val_a

Epoch: 0241 train_loss= 9.41452 train_acc= 0.23363 val_loss= 17.50740 val_acc= 0.38303 time= 10.10352
Epoch: 0242 train_loss= 7.88681 train_acc= 0.24775 val_loss= 17.48282 val_acc= 0.38303 time= 10.10715
Epoch: 0243 train_loss= 9.22976 train_acc= 0.21566 val_loss= 17.43946 val_acc= 0.38432 time= 10.08175
Epoch: 0244 train_loss= 9.06951 train_acc= 0.23620 val_loss= 17.41250 val_acc= 0.38560 time= 10.11469
Epoch: 0245 train_loss= 8.34277 train_acc= 0.21309 val_loss= 17.38198 val_acc= 0.38560 time= 10.01817
Epoch: 0246 train_loss= 10.15687 train_acc= 0.24519 val_loss= 17.37591 val_acc= 0.39075 time= 10.01461
Epoch: 0247 train_loss= 7.10354 train_acc= 0.22978 val_loss= 17.37362 val_acc= 0.39075 time= 10.08671
Epoch: 0248 train_loss= 9.65806 train_acc= 0.25802 val_loss= 17.35454 val_acc= 0.39203 time= 10.04643
Epoch: 0249 train_loss= 5.31145 train_acc= 0.23235 val_loss= 17.33066 val_acc= 0.39075 time= 10.02977
Epoch: 0250 train_loss= 7.19428 train_acc= 0.26829 val_loss= 17.30233 val_acc= 0.

Epoch: 0322 train_loss= 6.17212 train_acc= 0.21951 val_loss= 13.49052 val_acc= 0.32905 time= 11.03098
Epoch: 0323 train_loss= 7.59732 train_acc= 0.22080 val_loss= 13.46160 val_acc= 0.33033 time= 10.10971
Epoch: 0324 train_loss= 5.29685 train_acc= 0.25417 val_loss= 13.43024 val_acc= 0.32905 time= 10.03883
Epoch: 0325 train_loss= 6.13692 train_acc= 0.26059 val_loss= 13.39267 val_acc= 0.33162 time= 10.15017
Epoch: 0326 train_loss= 8.13469 train_acc= 0.23235 val_loss= 13.37054 val_acc= 0.33290 time= 10.13595
Epoch: 0327 train_loss= 6.43751 train_acc= 0.22208 val_loss= 13.34554 val_acc= 0.33290 time= 10.05798
Epoch: 0328 train_loss= 6.70601 train_acc= 0.23363 val_loss= 13.32617 val_acc= 0.33162 time= 10.02800
Epoch: 0329 train_loss= 7.49256 train_acc= 0.22850 val_loss= 13.30678 val_acc= 0.33290 time= 10.05106
Epoch: 0330 train_loss= 7.13865 train_acc= 0.24134 val_loss= 13.27764 val_acc= 0.33290 time= 10.10111
Epoch: 0331 train_loss= 5.86257 train_acc= 0.21181 val_loss= 13.25172 val_acc= 0.3

Epoch: 0403 train_loss= 6.01022 train_acc= 0.22336 val_loss= 11.41377 val_acc= 0.30848 time= 10.07595
Epoch: 0404 train_loss= 4.85069 train_acc= 0.22721 val_loss= 11.38272 val_acc= 0.30848 time= 10.06347
Epoch: 0405 train_loss= 4.20779 train_acc= 0.22850 val_loss= 11.35916 val_acc= 0.30977 time= 10.05415
Epoch: 0406 train_loss= 6.03339 train_acc= 0.21181 val_loss= 11.32777 val_acc= 0.31105 time= 10.18140
Epoch: 0407 train_loss= 5.40622 train_acc= 0.23363 val_loss= 11.29851 val_acc= 0.31234 time= 10.04509
Epoch: 0408 train_loss= 6.66147 train_acc= 0.22593 val_loss= 11.26594 val_acc= 0.31491 time= 10.03931
Epoch: 0409 train_loss= 5.67104 train_acc= 0.23363 val_loss= 11.23388 val_acc= 0.31748 time= 10.08884
Epoch: 0410 train_loss= 5.56684 train_acc= 0.26573 val_loss= 11.20254 val_acc= 0.32134 time= 10.03471
Epoch: 0411 train_loss= 5.40521 train_acc= 0.24134 val_loss= 11.17568 val_acc= 0.32134 time= 10.14120
Epoch: 0412 train_loss= 6.21449 train_acc= 0.23363 val_loss= 11.14515 val_acc= 0.3

Epoch: 0484 train_loss= 3.84775 train_acc= 0.24775 val_loss= 10.32338 val_acc= 0.32005 time= 10.77939
Epoch: 0485 train_loss= 4.40721 train_acc= 0.25417 val_loss= 10.30548 val_acc= 0.32648 time= 11.00741
Epoch: 0486 train_loss= 3.64997 train_acc= 0.24390 val_loss= 10.28928 val_acc= 0.33033 time= 11.82871
Epoch: 0487 train_loss= 4.14216 train_acc= 0.22593 val_loss= 10.27365 val_acc= 0.33290 time= 11.15288
Epoch: 0488 train_loss= 4.58071 train_acc= 0.24904 val_loss= 10.25434 val_acc= 0.33548 time= 10.26356
Epoch: 0489 train_loss= 3.87467 train_acc= 0.21053 val_loss= 10.23793 val_acc= 0.33548 time= 10.51335
Epoch: 0490 train_loss= 4.23529 train_acc= 0.25417 val_loss= 10.21700 val_acc= 0.33548 time= 10.16824
Epoch: 0491 train_loss= 4.78875 train_acc= 0.25802 val_loss= 10.19701 val_acc= 0.33548 time= 10.09219
Epoch: 0492 train_loss= 4.67677 train_acc= 0.25160 val_loss= 10.16925 val_acc= 0.33548 time= 10.29424
Epoch: 0493 train_loss= 3.94735 train_acc= 0.25546 val_loss= 10.14841 val_acc= 0.3

In [ ]:
# Testing
test_cost, test_acc, test_duration = evaluate(features, support, test_labels_mat, test_mask_mat[:,1], placeholders, sess, model)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))